In [1]:
import timeit
import pandas as pd

import warnings
warnings.simplefilter(action='ignore')

from tsfresh import extract_features
from dask.distributed import Client
from tsfresh.utilities.distribution import ClusterDaskDistributor

In [ ]:
client = Client()
ip = client.scheduler.address.split('//')[-1]
client

In [2]:
CO_timeseries_featurized_prepath = '/home/ubuntu/workspace_rohan/project/data/processed/'
CO_timeseries_path = '/home/ubuntu/workspace_rohan/project/data/raw/CO_timeseries.csv'
CO_timeseries_unpivot_path = '/home/ubuntu/workspace_rohan/project/data/processed/CO_timeseries_unpivot.csv'

CO_timeseries_unpivot_shared_prepath = '/home/ubuntu/workspace_rohan/project/data/processed/CO_timeseries_unpivot_shared/'

CO_timeseries_unpivot_featurized_shared_prepath = '/home/ubuntu/workspace_rohan/project/data/processed/CO_timeseries_unpivot_featurized_shared/'

CO_processed_path = '/home/ubuntu/workspace_rohan/project/data/raw/CO_processed.csv'

In [3]:
df_pd = pd.read_csv(CO_timeseries_unpivot_path)
df_pd = df_pd[['index', 'week_int', 'week_value']]

In [4]:
th = round(df_pd.shape[0]/19)
step = 4000

In [ ]:
start = 0
while (start < th):
    end = start+step
    print(start, end)
    df_temp = df_pd[(df_pd['index']>=start) & (df_pd['index']<end)]
    df_temp.to_csv(CO_timeseries_unpivot_shared_prepath+'CO_timeseries_unpivot_shared_'+str(start)+'_'+str(end)+'.csv', index=False)
    start = start+step

In [ ]:
start = 0
while (start < th):
    end = start+step
    print(start, end)
    df_temp = pd.read_csv(CO_timeseries_unpivot_shared_prepath+'CO_timeseries_unpivot_shared_'+str(start)+'_'+str(end)+'.csv')
    Distributor = ClusterDaskDistributor(address=ip)
    X = extract_features(timeseries_container=df_temp,
                         column_id='index', column_sort='week_int',
                         disable_progressbar=False,
                         distributor=Distributor)
    
    X.to_csv(CO_timeseries_unpivot_featurized_shared_prepath+'CO_timeseries_unpivot_featurized_shared_'+str(start)+'_'+str(end)+'.csv', index=False)
    start = start+step

In [5]:
data = pd.read_csv(CO_processed_path)
data = data.drop(["Unnamed: 0"],axis=1)
data.normalized_rotation.fillna(-1, inplace=True)
data.normalized_freq.fillna(-1, inplace=True)
na_check = data.columns

In [6]:
standardize_columns = ['week_1',
       'week_2', 'week_3', 'week_4', 'week_5', 'week_6', 'week_7', 'week_8',
       'week_9', 'week_10', 'week_11', 'week_12', 'week_13', 'week_14',
       'week_15', 'week_16', 'week_17', 'week_18', 'week_19']

df_CO_timeseries_raw = pd.read_csv(CO_timeseries_path)
df_CO_timeseries_raw.drop(['Unnamed: 0'], axis=1, inplace=True)
df_CO_timeseries_featurized = pd.DataFrame()
start = 0
while (start < th):
    
    end = start+step
    print(start, end)
    df_temp = df_CO_timeseries_raw.iloc[start:end, :]
    df_temp.reset_index(inplace=True)
    
    df_feat = pd.read_csv(CO_timeseries_unpivot_featurized_shared_prepath+'CO_timeseries_unpivot_featurized_shared_'+str(start)+'_'+str(end)+'.csv')    
    df_temp_full = pd.concat([df_temp, df_feat], axis=1)
    
    df_temp_new = df_temp_full[standardize_columns]
    df_temp_new = df_temp_new.div(df_temp_new.max(axis=1), axis=0).fillna(0)
    df_temp_new.columns = [str(col) + '_standardized' for col in df_temp_new.columns]
    df_temp_full[list(df_temp_new.columns)] = df_temp_new
    
    df_temp_full = pd.merge(df_temp_full, data, how='left', left_on=['fecha_de_visita', 'codigo_de_cliente', 'codigo_de_producto'], right_on=['fecha_de_visita', 'codigo_de_cliente', 'codigo_de_producto'])
    df_temp_full = df_temp_full.dropna(subset=na_check)
    
    if start==0:
        df_temp_full.to_csv(CO_timeseries_featurized_prepath+'CO_timeseries_featurized.csv', mode='a', index=False)
    else:
        df_temp_full.to_csv(CO_timeseries_featurized_prepath+'CO_timeseries_featurized.csv', mode='a', header=False, index=False)
    start = start+step
    
    
    
    

0 4000
4000 8000
8000 12000
12000 16000
16000 20000
20000 24000
24000 28000
28000 32000
32000 36000
36000 40000
40000 44000
44000 48000
48000 52000
52000 56000
56000 60000
60000 64000
64000 68000
68000 72000
72000 76000
76000 80000
80000 84000
84000 88000
88000 92000
92000 96000
96000 100000
100000 104000
104000 108000
108000 112000
112000 116000
116000 120000
120000 124000
124000 128000
128000 132000
132000 136000
136000 140000
140000 144000
144000 148000
148000 152000
152000 156000
156000 160000
160000 164000
164000 168000
168000 172000
172000 176000
176000 180000
180000 184000
184000 188000
188000 192000
192000 196000
196000 200000
200000 204000
204000 208000
208000 212000
212000 216000
216000 220000
220000 224000
224000 228000
228000 232000
232000 236000
236000 240000
240000 244000
244000 248000
248000 252000
252000 256000
256000 260000
260000 264000
264000 268000
268000 272000
272000 276000
276000 280000
280000 284000
284000 288000
288000 292000
292000 296000
296000 300000
300000 

2188000 2192000
2192000 2196000
2196000 2200000
2200000 2204000
2204000 2208000
2208000 2212000
2212000 2216000
2216000 2220000
2220000 2224000
2224000 2228000
2228000 2232000
2232000 2236000
2236000 2240000
2240000 2244000
2244000 2248000
2248000 2252000
2252000 2256000
2256000 2260000
2260000 2264000
2264000 2268000
2268000 2272000
2272000 2276000
2276000 2280000
2280000 2284000
2284000 2288000
2288000 2292000
2292000 2296000
2296000 2300000
2300000 2304000
2304000 2308000
2308000 2312000
2312000 2316000
2316000 2320000
2320000 2324000
2324000 2328000
2328000 2332000
2332000 2336000
2336000 2340000
2340000 2344000
2344000 2348000
2348000 2352000
2352000 2356000
2356000 2360000
2360000 2364000
2364000 2368000
2368000 2372000
2372000 2376000
2376000 2380000
2380000 2384000
2384000 2388000
2388000 2392000
2392000 2396000
2396000 2400000
2400000 2404000
2404000 2408000
2408000 2412000
2412000 2416000
2416000 2420000
2420000 2424000
2424000 2428000
2428000 2432000
2432000 2436000
2436000 

Data dump, notebook copy, script copy, model objects pipeline on 8000 (+ dump), db 2 dumps

In [338]:
import pandas as pd
from xgboost import XGBClassifier
from imblearn.ensemble import BalancedRandomForestClassifier


report_path = '/home/ubuntu/workspace_rohan/project/reports/model_performance/work_v5.3_standardized.txt'
train_split_format_path = '/home/ubuntu/workspace_rohan/project/data/processed/model_data/train_split_format.csv'
train_heldout_format_path = '/home/ubuntu/workspace_rohan/project/data/processed/model_data/train_heldout_format.csv'
model_save_prepath = '/home/ubuntu/workspace_rohan/project/models/'

seed = 7

features = [
    'week_value__variance_larger_than_standard_deviation',
 'week_value__has_duplicate_max',
 'week_value__has_duplicate_min',
 'week_value__has_duplicate',
 'week_value__sum_values',
 'week_value__abs_energy',
 'week_value__mean_abs_change',
 'week_value__mean_change',
 'week_value__mean_second_derivative_central',
 'week_value__median',
 'week_value__mean',
 'week_value__length',
 'week_value__standard_deviation',
 'week_value__variation_coefficient',
 'week_value__variance',
 'week_value__skewness',
 'week_value__kurtosis',
 'week_value__root_mean_square',
 'week_value__absolute_sum_of_changes',
 'week_value__longest_strike_below_mean',
 'week_value__longest_strike_above_mean',
 'week_value__count_above_mean',
 'week_value__count_below_mean',

 'week_value__time_reversal_asymmetry_statistic__lag_1',
 'week_value__time_reversal_asymmetry_statistic__lag_2',
 'week_value__time_reversal_asymmetry_statistic__lag_3',
 'week_value__c3__lag_1',
 'week_value__c3__lag_2',
 'week_value__c3__lag_3',
 'week_value__cid_ce__normalize_True',
 'week_value__cid_ce__normalize_False',
 'week_value__symmetry_looking__r_0.0',
 'week_value__symmetry_looking__r_0.05',
 'week_value__symmetry_looking__r_0.1',
 'week_value__symmetry_looking__r_0.15000000000000002',
 'week_value__symmetry_looking__r_0.2',
 'week_value__symmetry_looking__r_0.25',
 'week_value__symmetry_looking__r_0.30000000000000004',
 'week_value__symmetry_looking__r_0.35000000000000003',
 'week_value__symmetry_looking__r_0.4',
 'week_value__symmetry_looking__r_0.45',
 'week_value__symmetry_looking__r_0.5',
 'week_value__symmetry_looking__r_0.55',
 'week_value__symmetry_looking__r_0.6000000000000001',
 'week_value__symmetry_looking__r_0.65',
 'week_value__symmetry_looking__r_0.7000000000000001',
 'week_value__symmetry_looking__r_0.75',
 'week_value__symmetry_looking__r_0.8',
 'week_value__symmetry_looking__r_0.8500000000000001',
 'week_value__symmetry_looking__r_0.9',
 'week_value__symmetry_looking__r_0.9500000000000001',
 'week_value__large_standard_deviation__r_0.05',
 'week_value__large_standard_deviation__r_0.1',
 'week_value__large_standard_deviation__r_0.15000000000000002',
 'week_value__large_standard_deviation__r_0.2',
 'week_value__large_standard_deviation__r_0.25',
 'week_value__large_standard_deviation__r_0.30000000000000004',
 'week_value__large_standard_deviation__r_0.35000000000000003',
 'week_value__large_standard_deviation__r_0.4',
 'week_value__large_standard_deviation__r_0.45',
 'week_value__large_standard_deviation__r_0.5',
 'week_value__large_standard_deviation__r_0.55',
 'week_value__large_standard_deviation__r_0.6000000000000001',
 'week_value__large_standard_deviation__r_0.65',
 'week_value__large_standard_deviation__r_0.7000000000000001',
 'week_value__large_standard_deviation__r_0.75',
 'week_value__large_standard_deviation__r_0.8',
 'week_value__large_standard_deviation__r_0.8500000000000001',
 'week_value__large_standard_deviation__r_0.9',
 'week_value__large_standard_deviation__r_0.9500000000000001',
 'week_value__quantile__q_0.1',
 'week_value__quantile__q_0.2',
 'week_value__quantile__q_0.3',
 'week_value__quantile__q_0.4',
 'week_value__quantile__q_0.6',
 'week_value__quantile__q_0.7',
 'week_value__quantile__q_0.8',
 'week_value__quantile__q_0.9',
 'week_value__autocorrelation__lag_0',
 'week_value__autocorrelation__lag_1',
 'week_value__autocorrelation__lag_2',
 'week_value__autocorrelation__lag_3',
 'week_value__autocorrelation__lag_4',
 'week_value__autocorrelation__lag_5',
 'week_value__autocorrelation__lag_6',
 'week_value__autocorrelation__lag_7',
 'week_value__autocorrelation__lag_8',
 'week_value__autocorrelation__lag_9',
 'week_value__agg_autocorrelation__f_agg_"mean"__maxlag_40',
 'week_value__agg_autocorrelation__f_agg_"median"__maxlag_40',
 'week_value__agg_autocorrelation__f_agg_"var"__maxlag_40',

 'week_value__number_cwt_peaks__n_1',
 'week_value__number_cwt_peaks__n_5',
 'week_value__number_peaks__n_1',
 'week_value__number_peaks__n_3',
 'week_value__number_peaks__n_5',
 'week_value__number_peaks__n_10',
 'week_value__number_peaks__n_50',
 'week_value__binned_entropy__max_bins_10',
 'week_value__index_mass_quantile__q_0.1',
 'week_value__index_mass_quantile__q_0.2',
 'week_value__index_mass_quantile__q_0.3',
 'week_value__index_mass_quantile__q_0.4',
 'week_value__index_mass_quantile__q_0.6',
 'week_value__index_mass_quantile__q_0.7',
 'week_value__index_mass_quantile__q_0.8',
 'week_value__index_mass_quantile__q_0.9',
 'week_value__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_0__w_10__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_1__w_2__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_1__w_5__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_1__w_10__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_1__w_20__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_2__w_2__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_2__w_5__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_2__w_10__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_2__w_20__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_3__w_2__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_3__w_5__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_3__w_10__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_3__w_20__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_4__w_5__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_4__w_10__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_4__w_20__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_5__w_5__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_5__w_10__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_5__w_20__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_6__w_5__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_6__w_10__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_6__w_20__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_7__w_2__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_7__w_5__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_7__w_10__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_7__w_20__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_8__w_2__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_8__w_5__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_8__w_10__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_8__w_20__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_9__w_2__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_9__w_5__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_9__w_10__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_9__w_20__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_10__w_2__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_10__w_5__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_10__w_10__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_10__w_20__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_11__w_2__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_11__w_5__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_11__w_10__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_11__w_20__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_12__w_2__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_12__w_5__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_12__w_10__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_12__w_20__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_13__w_2__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_13__w_5__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_13__w_10__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_13__w_20__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_14__w_2__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_14__w_5__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_14__w_10__widths_(2, 5, 10, 20)',
 'week_value__cwt_coefficients__coeff_14__w_20__widths_(2, 5, 10, 20)',
 'week_value__spkt_welch_density__coeff_2',
 'week_value__spkt_welch_density__coeff_5',
 'week_value__spkt_welch_density__coeff_8',
 'week_value__ar_coefficient__coeff_0__k_10',
 'week_value__ar_coefficient__coeff_1__k_10',
 'week_value__ar_coefficient__coeff_2__k_10',
 'week_value__ar_coefficient__coeff_3__k_10',
 'week_value__ar_coefficient__coeff_4__k_10',
 'week_value__ar_coefficient__coeff_5__k_10',
 'week_value__ar_coefficient__coeff_6__k_10',
 'week_value__ar_coefficient__coeff_7__k_10',
 'week_value__ar_coefficient__coeff_8__k_10',
 'week_value__ar_coefficient__coeff_9__k_10',
 'week_value__ar_coefficient__coeff_10__k_10',
 'week_value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.2__ql_0.0',
 'week_value__change_quantiles__f_agg_"var"__isabs_False__qh_0.2__ql_0.0',
 'week_value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.2__ql_0.0',
 'week_value__change_quantiles__f_agg_"var"__isabs_True__qh_0.2__ql_0.0',
 'week_value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.4__ql_0.0',
 'week_value__change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.0',
 'week_value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.0',
 'week_value__change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.0',
 'week_value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.0',
 'week_value__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.0',
 'week_value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.0',
 'week_value__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.0',
 'week_value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.0',
 'week_value__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.0',
 'week_value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.0',
 'week_value__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.0',
 'week_value__change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.0',
 'week_value__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.0',
 'week_value__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.0',
 'week_value__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.0',
 'week_value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.4__ql_0.2',
 'week_value__change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.2',
 'week_value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.2',
 'week_value__change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.2',
 'week_value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.2',
 'week_value__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.2',
 'week_value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.2',
 'week_value__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.2',
 'week_value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.2',
 'week_value__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.2',
 'week_value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.2',
 'week_value__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.2',
 'week_value__change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.2',
 'week_value__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.2',
 'week_value__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.2',
 'week_value__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.2',
 'week_value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.4',
 'week_value__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.4',
 'week_value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.4',
 'week_value__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.4',
 'week_value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.4',
 'week_value__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.4',
 'week_value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.4',
 'week_value__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.4',
 'week_value__change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.4',
 'week_value__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.4',
 'week_value__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.4',
 'week_value__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.4',
 'week_value__change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.6',
 'week_value__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.6',
 'week_value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.6',
 'week_value__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.6',
 'week_value__change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.6',
 'week_value__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.6',
 'week_value__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.6',
 'week_value__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.6',
 'week_value__change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.8',
 'week_value__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.8',
 'week_value__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.8',
 'week_value__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.8',



 
 'week_value__fft_aggregated__aggtype_"centroid"',
 'week_value__fft_aggregated__aggtype_"variance"',
 'week_value__fft_aggregated__aggtype_"skew"',
 'week_value__fft_aggregated__aggtype_"kurtosis"',
 'week_value__value_count__value_0',
 'week_value__value_count__value_1',
 'week_value__value_count__value_-1',
 'week_value__range_count__max_1__min_-1',
 'week_value__range_count__max_0__min_1000000000000.0',
 'week_value__range_count__max_1000000000000.0__min_0',
 'week_value__approximate_entropy__m_2__r_0.1',
 'week_value__approximate_entropy__m_2__r_0.3',
 'week_value__approximate_entropy__m_2__r_0.5',
 'week_value__approximate_entropy__m_2__r_0.7',
 'week_value__approximate_entropy__m_2__r_0.9',

    'week_value__number_crossing_m__m_0',
 'week_value__number_crossing_m__m_-1',
 'week_value__number_crossing_m__m_1',
    
    
 'week_value__energy_ratio_by_chunks__num_segments_10__segment_focus_0',
 'week_value__energy_ratio_by_chunks__num_segments_10__segment_focus_1',
 'week_value__energy_ratio_by_chunks__num_segments_10__segment_focus_2',
 'week_value__energy_ratio_by_chunks__num_segments_10__segment_focus_3',
 'week_value__energy_ratio_by_chunks__num_segments_10__segment_focus_4',
 'week_value__energy_ratio_by_chunks__num_segments_10__segment_focus_5',
 'week_value__energy_ratio_by_chunks__num_segments_10__segment_focus_6',
 'week_value__energy_ratio_by_chunks__num_segments_10__segment_focus_7',
 'week_value__energy_ratio_by_chunks__num_segments_10__segment_focus_8',
 'week_value__energy_ratio_by_chunks__num_segments_10__segment_focus_9',
 'week_value__ratio_beyond_r_sigma__r_0.5',
 'week_value__ratio_beyond_r_sigma__r_1',
 'week_value__ratio_beyond_r_sigma__r_1.5',
 'week_value__ratio_beyond_r_sigma__r_2',
 'week_value__ratio_beyond_r_sigma__r_2.5',
 'week_value__ratio_beyond_r_sigma__r_3',
 'week_value__ratio_beyond_r_sigma__r_5',
 'week_value__ratio_beyond_r_sigma__r_6',
 'week_value__ratio_beyond_r_sigma__r_7',
 'week_value__ratio_beyond_r_sigma__r_10',
 'week_value__count_above__t_0',
 'week_value__count_below__t_0',
 'week_value__lempel_ziv_complexity__bins_2',
 'week_value__lempel_ziv_complexity__bins_3',
 'week_value__lempel_ziv_complexity__bins_5',
 'week_value__lempel_ziv_complexity__bins_10',
 'week_value__lempel_ziv_complexity__bins_100',
 'week_value__fourier_entropy__bins_2',
 'week_value__fourier_entropy__bins_3',
 'week_value__fourier_entropy__bins_5',
 'week_value__fourier_entropy__bins_10',
 'week_value__fourier_entropy__bins_100',
 'week_value__permutation_entropy__dimension_3__tau_1',
 'week_value__permutation_entropy__dimension_4__tau_1',
 'week_value__permutation_entropy__dimension_5__tau_1',
 'week_value__permutation_entropy__dimension_6__tau_1',
 'week_value__permutation_entropy__dimension_7__tau_1',

            
            'week_1_standardized', 'week_2_standardized', 'week_3_standardized',
       'week_4_standardized', 'week_5_standardized', 'week_6_standardized',
       'week_7_standardized', 'week_8_standardized', 'week_9_standardized',
       'week_10_standardized', 'week_11_standardized', 'week_12_standardized',
       'week_13_standardized', 'week_14_standardized', 'week_15_standardized',
       'week_16_standardized', 'week_17_standardized', 'week_18_standardized',
       'week_19_standardized', 'cod_canal', 'cod_giro', 'cod_subgiro',
       'desc_region', 'desc_subregion', 'desc_division', 'cod_zona', 'ruta',
       'cod_modulo', 'categoria', 'marca', 'desc_sabor', 'desc_tipoenvase',
       'desc_subfamilia', 'contenido',
       'product_sales_amount_last_3m', 'product_trnx_last_3m',
       'normalized_rotation', 'normalized_freq', 'total_sales_last_3m',
       'total_trnx_last_3m', 'ratio_sales_last_3m', 'ratio_trnx_last_3m',
       'bought_last_year_flag', 'prod_coverage_bucket']





In [339]:
from psycopg2 import connect

In [340]:
train_split_format_path = '/home/ubuntu/workspace_rohan/project/data/processed/model_data/train_split_format.csv'
train_heldout_format_path = '/home/ubuntu/workspace_rohan/project/data/processed/model_data/train_heldout_format.csv'
test_split_format_path = '/home/ubuntu/workspace_rohan/project/data/processed/model_data/test_split_format.csv'
test_heldout_format_path = '/home/ubuntu/workspace_rohan/project/data/processed/model_data/test_heldout_format.csv'

In [341]:
model = XGBClassifier()
model.load_model(model_save_prepath+"xgboost_without_optuna_split_format.json")
test = pd.read_csv(test_split_format_path)
test.fillna(0, inplace=True)


In [342]:
test['prediction probability'] = model.predict_proba(test[features])[:,1:]
prediction_table = test[['codigo_de_cliente', 'fecha_de_visita', 'codigo_de_producto', 'prediction probability', 'bought_in_the_visit']].sort_values(['codigo_de_cliente', 'fecha_de_visita', 'prediction probability'], ascending=False)
prediction_table['rank'] = 1
prediction_table['rank'] = prediction_table.groupby(['codigo_de_cliente', 'fecha_de_visita'])['rank'].cumsum()


In [343]:
HOST = 'localhost'
USER = 'postgres'
PASSWORD = 'Y3DjpNRR'
DATABASE = 'bi_db_aje_co'

In [344]:
prediction_table.head()

,codigo_de_cliente,fecha_de_visita,codigo_de_producto,prediction probability,bought_in_the_visit,rank
4548,94176284,2020-12-04,500040,0.915200,0.0,1
3114,94176284,2020-12-04,501177,0.065982,0.0,2
2970,94176284,2020-12-04,510052,0.045180,0.0,3
7819,94176284,2020-12-04,508720,0.024756,0.0,4
5469,94176284,2020-12-04,500593,0.008908,0.0,5


In [345]:
prediction_table['fecha_de_visita'] = prediction_table.fecha_de_visita.astype(str)
prediction_table['codigo_de_cliente'] = prediction_table.codigo_de_cliente.astype(str)
prediction_table['codigo_de_producto'] = prediction_table.codigo_de_producto.astype(str)

In [346]:
prediction_table['codigo_de_cliente_new'] = '00' + prediction_table.codigo_de_cliente.astype(str)

In [347]:
prediction_table.head(2)

,codigo_de_cliente,fecha_de_visita,codigo_de_producto,prediction probability,bought_in_the_visit,rank,codigo_de_cliente_new
4548,94176284,2020-12-04,500040,0.915200,0.0,1,0094176284
3114,94176284,2020-12-04,501177,0.065982,0.0,2,0094176284


In [348]:
clients = list(prediction_table.codigo_de_cliente.unique())
clients = ['00'+str(x) for x in clients]
client_list = '(\'' + '\', \''.join(clients) + '\')'
query = '''
select
    fecha_de_visita,
    codigo_de_cliente,
    recomm_qt
from public.regular_recomm_qty_dtl
where codigo_de_cliente in ''' + client_list + ''';
'''

In [350]:
conn = connect(
        host=HOST,
        database=DATABASE,
        user=USER,
        password=PASSWORD
    )

In [351]:
df = pd.read_sql(query, conn, coerce_float=False)
df.recomm_qt.fillna(0, inplace=True)
df['recomm_qt'] = df['recomm_qt'].astype(int)
df['fecha_de_visita'] = df.fecha_de_visita.astype(str)
df['codigo_de_cliente'] = df.codigo_de_cliente.astype(str)

In [352]:
df.head()

,fecha_de_visita,codigo_de_cliente,recomm_qt
0,2020-09-10,0094100001,0
1,2020-10-02,0094100001,2
2,2020-10-10,0094100001,2
3,2020-10-23,0094100001,2
4,2020-10-24,0094100001,2


In [353]:
prediction_table = pd.merge(prediction_table, df, how='left', left_on=['codigo_de_cliente_new', 'fecha_de_visita'], right_on=['codigo_de_cliente', 'fecha_de_visita'])


In [355]:
prediction_table.rename({
    'codigo_de_cliente_x' : 'codigo_de_cliente'
}, inplace=True, axis=1)

prediction_table.drop(['codigo_de_cliente_y'], inplace=True, axis=1)

In [357]:
prediction_table = prediction_table[prediction_table['rank'] <= prediction_table['recomm_qt']]

In [364]:
prediction_table.shape

(4282, 8)

In [360]:
prediction_table.head(5)

,codigo_de_cliente,fecha_de_visita,codigo_de_producto,prediction probability,bought_in_the_visit,rank,codigo_de_cliente_new,recomm_qt
0,94176284,2020-12-04,500040,0.915200,0.0,1,0094176284,2
1,94176284,2020-12-04,501177,0.065982,0.0,2,0094176284,2
5,94176217,2020-12-05,500290,0.228075,1.0,1,0094176217,2
6,94176217,2020-12-05,500570,0.078977,1.0,2,0094176217,2
9,94176205,2020-12-03,508524,0.138301,0.0,1,0094176205,2


In [361]:
df_sales = pd.read_sql("select fecha_de_visita, codigo_de_cliente, codigo_de_producto, venta_neta_dolar "
                           "from public.sales where venta_neta_dolar > 0.00", conn, coerce_float=False)

In [362]:
df_sales.fecha_de_visita = df_sales.fecha_de_visita.astype(str)
df_sales.codigo_de_cliente = df_sales.codigo_de_cliente.astype(str)
df_sales.codigo_de_producto = df_sales.codigo_de_producto.astype(str)

In [363]:
df_sales.head()

,fecha_de_visita,codigo_de_cliente,codigo_de_producto,venta_neta_dolar
0,2021-01-12,00945097,501323,111600.00
1,2021-03-11,009460171,509603,8550.00
2,2020-07-15,0094173605,509603,8037.00
3,2020-10-24,0094114343,509641,3450.00
4,2020-11-23,00941107,515512,14400.00


In [365]:
prediction_table = pd.merge(prediction_table, df_sales, how='left', left_on=['codigo_de_cliente_new', 'fecha_de_visita', 'codigo_de_producto'], right_on=['codigo_de_cliente', 'fecha_de_visita', 'codigo_de_producto'])


In [371]:
prediction_table.rename({
    'codigo_de_cliente_x' : 'codigo_de_cliente'
}, inplace=True, axis=1)


In [373]:
prediction_table['venta_neta_dolar'] = prediction_table['venta_neta_dolar'].fillna(0.00)

In [375]:
prediction_table['executed_flag'] = prediction_table['venta_neta_dolar'].apply(lambda x: 1 if x > 0.00 else 0)

In [378]:
client_level_precision = prediction_table.groupby(['fecha_de_visita', 'codigo_de_cliente']).agg({'codigo_de_producto': 'count', 'executed_flag': 'sum'}).reset_index()


In [379]:
client_level_precision

,fecha_de_visita,codigo_de_cliente,codigo_de_producto,executed_flag
0,2020-12-01,94100005,2,0
1,2020-12-01,94101752,1,1
2,2020-12-01,94102195,2,0
3,2020-12-01,94103299,3,1
4,2020-12-01,94103960,2,0
...,...,...,...,...
1770,2020-12-06,94172473,3,1
1771,2020-12-06,94172487,1,0
1772,2020-12-06,94173070,1,0
1773,2020-12-06,94173237,2,1


In [380]:
client_level_precision['precision'] = client_level_precision['executed_flag']/client_level_precision['codigo_de_producto']


In [381]:
client_level_precision

,fecha_de_visita,codigo_de_cliente,codigo_de_producto,executed_flag,precision
0,2020-12-01,94100005,2,0,0.000000
1,2020-12-01,94101752,1,1,1.000000
2,2020-12-01,94102195,2,0,0.000000
3,2020-12-01,94103299,3,1,0.333333
4,2020-12-01,94103960,2,0,0.000000
...,...,...,...,...,...
1770,2020-12-06,94172473,3,1,0.333333
1771,2020-12-06,94172487,1,0,0.000000
1772,2020-12-06,94173070,1,0,0.000000
1773,2020-12-06,94173237,2,1,0.500000


In [382]:
precision = client_level_precision['precision'].mean()

In [383]:
precision

0.3400027437351381

In [376]:
prediction_table.head()

,codigo_de_cliente,fecha_de_visita,codigo_de_producto,prediction probability,bought_in_the_visit,rank,codigo_de_cliente_new,recomm_qt,codigo_de_cliente_y,venta_neta_dolar,executed_flag
0,94176284,2020-12-04,500040,0.915200,0.0,1,0094176284,2,NaN,0,0
1,94176284,2020-12-04,501177,0.065982,0.0,2,0094176284,2,NaN,0,0
2,94176217,2020-12-05,500290,0.228075,1.0,1,0094176217,2,0094176217,21100.00,1
3,94176217,2020-12-05,500570,0.078977,1.0,2,0094176217,2,0094176217,21100.00,1
4,94176205,2020-12-03,508524,0.138301,0.0,1,0094176205,2,NaN,0,0
